In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [2]:
#load document
def doc_loader(file):
    from langchain.document_loaders import PyPDFLoader
    print(f'loading File..')
    loader = PyPDFLoader(file)
    data = loader.load()
    print(f'Loading comeplete!')
    return data

In [3]:
#load residential act
resi_act = doc_loader('Rental_act.pdf')

loading File..
Loading comeplete!


In [4]:
print(resi_act[0].page_content)

1 Français  
Residential Tenancies Act, 2006  
S.O. 2006 , CHAPTER 17 
Consolidation Period:  From June 8, 2023  to the e-Laws currenc y date. 
Last amendment:  2023, c. 10, Sched. 7 . 
Legislative History: 2006, c. 17, s. 261 ; 2006, c. 32, Sched. C, s. 56 ; 2006, c. 32, Sched. E, s. 7 ; 2006, c. 35, Sched. C, s. 118 ; 2007, c. 8, s. 
226; 2007,  c. 13, s. 48 ; 2008, c. 14, s. 58 ; 2009, c. 33, Sched. 8, s. 15 ; 2009, c. 33, Sched. 18,  s. 30; 2009, c. 33, Sched.  21, s. 11 ; 2010, c. 8, 
s. 39 ; 2011, c. 6, Sched. 1, s. 188 ; 2011, c. 6, Sched. 3 ; 2012, c. 6 ; 2013, c.  3, s. 20 -56; 2015, c. 38, Sched. 7, s. 60 ; 2016, c. 2, Sched. 6 ; 
2016, c. 23, s. 66 ; 2016, c. 25, Sched. 5 ; 2017, c. 13 ; 2017, c. 14, Sched. 4, s. 33 ; 2017, c. 25, Sched. 9, s. 116  (see 2023, c. 4, Sched. 1, s. 
67); 2017, c. 34, Sched. 46, s. 50 ; 2018, c. 6, Sched. 3,  s. 12 ; 2018, c. 17, Sched. 36 ; 2019, c. 7, Sched. 17 , s. 153 ; 2020, c. 1, s. 36 ; 2020, c. 
16, Sched. 3, s. 12 ; 2020, c. 16, Sched. 4

In [5]:
print('Number of pages: ', end='')
print(len(resi_act))

Number of pages: 130


In [6]:
def split_chunks(data):
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    text_split = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 0)
    chunks = text_split.split_documents(data)

    return chunks

In [7]:
chunks = split_chunks(resi_act)

In [8]:
print(len(chunks))

1128


In [9]:
#Cost of the embedding function
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    #total_tokens = sum([len(enc.encode(page.page_content))for page in texts])
    total_tokens = 0
    for page in texts:
        total_tokens += len(enc.encode(page.page_content))

        
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens /1000 * 0.0004:.6f}')

In [10]:
#total cost
print_embedding_cost(chunks)

Total Tokens: 152832
Embedding Cost in USD: 0.061133


In [11]:
#importing pinecone api keys
import pinecone
pinecone.init(api_key=os.getenv('PINECONE_APY_KEY'), environment=os.getenv('PINECONE_ENV'))

c:\Users\ankvi\anaconda3\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [12]:
def delete_pinecone_index(index_name='all'):
    
    import pinecone
    pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), 
                  environment=os.getenv('PINECONE_ENV'))

    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes...')
        for index in indexes:
            pinecone.delete_index(index)
        print('Done')
    else:
        print(f'Deleting {index_name}..', end='')
        pinecone.delete_index(index_name)
        print('{index_name} deleted')

In [13]:
delete_pinecone_index()

Deleting all indexes...
Done


In [14]:
pinecone.list_indexes()

[]

In [15]:
pinecone.create_index('residentact', dimension = 1536, metric='cosine')

In [16]:
pinecone.list_indexes()[0]

'residentact'

In [17]:
#convert chunks to embeddings and store it in pinecone database(vector database)

from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

def insert_fetch_embeddings():
    embeddings = OpenAIEmbeddings()
    vector_store = Pinecone.from_documents(chunks, embedding=embeddings, index_name=pinecone.list_indexes()[0])

    return vector_store


In [18]:
vector_store = insert_fetch_embeddings()

In [22]:
#code to retrive the vector embeddings from the pinecone database

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
pinecone.init(api_key=os.getenv('PINECONE_APY_KEY'), environment=os.getenv('PINECONE_ENV'))
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

index_name = pinecone.list_indexes()[0]
embeddings = OpenAIEmbeddings()

vectorstore = Pinecone.from_existing_index(index_name,embeddings)

In [23]:
pinecone.describe_index('residentact')

IndexDescription(name='residentact', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [28]:
def get_existing_vector():
    index_name = pinecone.list_indexes()[0]
    embeddings = OpenAIEmbeddings()
    vectorstore = Pinecone.from_existing_index(index_name,embeddings)

    return vectorstore

In [24]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


def ques_answer(vector_store, query):
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
    retriever = vector_store.as_retriever(search='similarity', search_kwargs = {'k' : 5})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)
    answer = chain.run(query)

    return answer

In [29]:
question ='Why was the residential act created?'
question1 = 'I have dog at my rental place but my lease agreement specifies no pets allowed, How do I prevent eviction if my landlord finds about my dog?'
print(ques_answer(get_existing_vector(),question1))

According to the information provided, a provision in the tenancy agreement prohibiting the presence of animals in the residential complex is void. However, the landlord may apply to the Board for an order terminating the tenancy and evicting the tenant if certain conditions are met. These conditions include if the past behavior of the animal substantially interferes with the reasonable enjoyment of the residential complex or if the presence of the animal causes the landlord or another tenant to suffer a serious allergic reaction. 

To prevent eviction if your landlord finds out about your dog, you would need to ensure that your dog's behavior does not substantially interfere with the reasonable enjoyment of the residential complex and does not cause any serious allergic reactions. If you can demonstrate that your dog is not causing these issues, the Board should not make an order terminating the tenancy based on these grounds. However, it is recommended to speak with a legal professio